## Question 2

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from math import sqrt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.regularizers import l2
from keras.layers import Dense, Dropout
from time import time

In [ ]:
filename1 = '/content/drive/My Drive/Machine Learning Homework 3/Q2_Test_Data.csv'
filename2 = '/content/drive/My Drive/Machine Learning Homework 3/Q2_Train_Data.csv'
filename3 = '/content/drive/My Drive/Machine Learning Homework 3/Q2_Validation_Data.csv'
df_test = pd.read_csv(filename1)
df_train = pd.read_csv(filename2)
df_validation = pd.read_csv(filename3)

In [ ]:
print(df_test.shape)
print(df_train.shape)
print(df_validation.shape)


In [ ]:
input_strings = df_train.pixels[50]
arr = np.array(input_strings.split(), dtype=int)
print(arr.shape)
X = arr.reshape(48, 48)
plt.imshow(X) 
plt.show() 

# get two visualization per emotion

In [ ]:
#@title
emotions_dict = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}
l = df_train.emotion.unique().size
df_temp = pd.DataFrame()
for emotion_number in range(0,l):
  print('Emotion = ', emotions_dict[emotion_number].upper())
  df_temp = df_train.loc[df_train['emotion']==emotion_number]
  df_temp = df_temp.sample(n = 2)
  indexes = list(df_temp.index)
  for i in indexes:
    input_strings = df_temp.pixels[i]
    arr = np.array(input_strings.split(), dtype=int)
    X = arr.reshape(48, 48)
    plt.imshow(X) 
    plt.show() 

    





# Count the number of samples per emotion

In [ ]:
for emotion_number in emotions_dict:
  print('The number of sample for emotion {} is {}'.format(emotions_dict[emotion_number],df_train.loc[df_train['emotion']==emotion_number].shape[0]))


# you will use a feedforward neural network(FNN) (also called “multilayer perceptron”) to perform the emotion classification task

In [ ]:


#  Define a Feed-Forward Model with 2 hidden layers with dimensions 1152 and 576 Neurons
model = Sequential([
  Dense(2304, activation='relu', input_shape=(48*48,), name="first_hidden_layer"),
  Dense(2304//2, activation='relu', name="second_hidden_layer"),Dropout(0.25),
  Dense(7, activation='softmax'),
])
print(model.summary())


# compile model
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)


In [ ]:
# Compile model

def preprocess_image(test_list):
  p = 0
  for i in range(0, len(test_list)): 
    test_list[i] = float(test_list[i]) 
  list_to_return = []
  mean = sum((test_list)) / len(test_list) 
  variance = sum([((x - mean) ** 2) for x in test_list]) / len(test_list) 
  res = variance ** 0.5
  adj_std = max(res, 1.0/sqrt(len(test_list)))
  for x in test_list:
    p = (x - mean)/adj_std
    list_to_return.append(p)
  return list_to_return



def preprocess_data(df_train):
  #data preprocess
  df_train['Array_pixels'] = df_train['pixels'].str.split(' ')
  train_X = []
  train_X1 = df_train['Array_pixels']
  for l in train_X1:
    list_final = preprocess_image(l)
    train_X.append(list_final)
  train_X = np.asarray(train_X).astype('float32')
  train_Y = df_train['emotion']
  train_Y = np.asarray(train_Y)
  return train_X, train_Y





# training data accuracy


In [ ]:

train_X, train_Y = preprocess_data(df_train)
start = time()
performance = model.fit(train_X, to_categorical(train_Y), epochs=10, batch_size=256,)
print('Training time for this model is = ', time()-start)


# Validation data accuracy


In [ ]:
train_X, train_Y = preprocess_data(df_validation)
performance_2 = model.evaluate(train_X, to_categorical(train_Y))

# performance_2 = model.fit(train_X, to_categorical(train_Y), epochs=10, batch_size=256,)


In [ ]:
train_X, train_Y = preprocess_data(df_test)
performance_2 = model.evaluate(train_X, to_categorical(train_Y))


# plot loss

In [ ]:
def plot_loss_function(performance):
  plt.plot(performance.history['loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train'], loc='upper left')
  plt.show()

plot_loss_function(performance)

# Convolutional Neural Network

In [ ]:
# Compile model

def preprocess_image_cnn(test_list):
  p = 0
  for i in range(0, len(test_list)): 
    test_list[i] = float(test_list[i]) 
  list_to_return = []
  mean = sum((test_list)) / len(test_list) 
  variance = sum([((x - mean) ** 2) for x in test_list]) / len(test_list) 
  res = variance ** 0.5
  adj_std = max(res, 1.0/sqrt(len(test_list)))
  for x in test_list:
    p = (x - mean)/adj_std
    list_to_return.append(p)
  return list_to_return



def preprocess_data_cnn(df_train):
  #data preprocess
  df_train['Array_pixels'] = df_train['pixels'].str.split(' ')
  train_X = []
  train_X1 = df_train['Array_pixels']
  for l in train_X1:
    list_final = preprocess_image(l)
    train_X.append(list_final)
  train_X = np.asarray(train_X).astype('float32')
  train_Y = df_train['emotion']
  train_Y = np.asarray(train_Y)
  return train_X, train_Y




In [ ]:
#get train, validation and test data
train_X, train_Y = preprocess_data(df_train)
train_X = train_X.reshape(28709,48,48,1)
validation_X, validation_Y = preprocess_data(df_validation)
validation_X = validation_X.reshape(3589,48,48,1)
test_X, test_Y = preprocess_data(df_test)
test_X = test_X.reshape(3589,48,48,1)

In [ ]:
# we separate the feature layers from classifier layer to accomodate the later part of tutorial on Fine-tuning.
from keras.layers import Conv2D, Flatten, MaxPooling2D

# Define 2 groups of layers: features layer (convolutions) and classification layer
common_features = [Conv2D(32, kernel_size=3, activation='relu', input_shape=(48,48,1)),
            Conv2D(32, kernel_size=3, activation='relu'), 
            MaxPooling2D(pool_size=(2,2)),
            Conv2D(64, kernel_size=3, activation='relu'),
            Conv2D(64, kernel_size=3, activation='relu'),
            MaxPooling2D(pool_size=(2,2)), Flatten(),]
classifier = [Dense(512, activation='relu',kernel_regularizer=l2(0.01)), Dropout(0.25), Dense(7, activation='softmax'),]

cnn_model = Sequential(common_features+classifier)

print(cnn_model.summary())  # Compare number of parameteres against FFN
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'],)



start = time()
cnn_model.fit(train_X, to_categorical(train_Y), epochs=10, batch_size=256,)
print('Training time for this model is = ', time()-start)

# Acc
performance = cnn_model.evaluate(validation_X, to_categorical(validation_Y))
print("Accuracy on Validation samples: {0}".format(performance[1]))

In [ ]:
cnn_model.evaluate(test_X, to_categorical(test_Y))

# Bayesian optimization for hyper-parameter tuning:

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials


def optimize_cnn(hyperparameter):
  
  # Define model using hyperparameters 
  cnn_model = Sequential([Conv2D(32, kernel_size=hyperparameter['conv_kernel_size'], activation='relu', input_shape=(48,48,1)), 
            Conv2D(32, kernel_size=hyperparameter['conv_kernel_size'], activation='relu'), 
            MaxPooling2D(pool_size=(2,2)), Dropout(hyperparameter['dropout_prob']),
            Conv2D(64, kernel_size=hyperparameter['conv_kernel_size'], activation='relu'),
            Conv2D(64, kernel_size=hyperparameter['conv_kernel_size'], activation='relu'), 
            MaxPooling2D(pool_size=(2,2)), Dropout(hyperparameter['dropout_prob']), 
            Flatten(),
            Dense(512, activation='relu', kernel_regularizer=l2(hyperparameter['lamda_value'])), 
            Dense(7, activation='softmax'),])
  
  cnn_model.compile(optimizer=hyperparameter['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'],)

  _ = cnn_model.fit(train_X, to_categorical(train_Y), epochs=2, batch_size=256, verbose=0)
  # Evaluate accuracy on validation data
  performance = cnn_model.evaluate(validation_X, to_categorical(validation_Y), verbose=0)

  print("Hyperparameters: ", hyperparameter, "Accuracy: ", performance[1])
  print("----------------------------------------------------")
  # We want to minimize loss i.e. negative of accuracy
  return({"status": STATUS_OK, "loss": -1*performance[1], "model":cnn_model})
  

# Define search space for hyper-parameters
space = {
    # The kernel_size for convolutions:
    'conv_kernel_size': hp.choice('conv_kernel_size', [1, 3, 5]),
    # Uniform distribution in finding appropriate dropout values
    'dropout_prob': hp.uniform('dropout_prob', 0.1, 0.35),
    # Choice of optimizer 
    'optimizer': hp.choice('optimizer', ['Adam', 'sgd']),
     # Uniform distribution in finding appropriate regularization values
    'lamda_value': hp.uniform('lamda_value', 0.01, 0.2),
}

trials = Trials()

# Find the best hyperparameters
best = fmin(
        optimize_cnn,
        space,
        algo=tpe.suggest,
        trials=trials,
        max_evals=50,
    )

print("==================================")
print("Best Hyperparameters", best)

# You can retrain the final model with optimal hyperparameters on train+validation data

# Or you can use the model returned directly
# Find trial which has minimum loss value and use that model to perform evaluation on the test data
test_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']

performance = test_model.evaluate(test_X, to_categorical(test_Y))

print("==================================")
print("Test Accuracy: ", performance[1])

# Fine tuning


In [ ]:
common_features = [Conv2D(32, kernel_size=3, activation='relu', input_shape=(48,48,1)), 
            Conv2D(32, kernel_size=3, activation='relu'), 
            MaxPooling2D(pool_size=(2,2)),
            Conv2D(64, kernel_size=3, activation='relu'),
            Conv2D(64, kernel_size=3, activation='relu'), 
            MaxPooling2D(pool_size=(2,2)), Flatten(),]

for l in common_features:
  l.trainable = False

classifier = [Dense(512, activation='relu',kernel_regularizer=l2(0.01)),Dropout(0.25), Dense(7, activation='softmax'),]
cnn_model = Sequential(common_features+classifier)
print(cnn_model.summary())
cnn_model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)

cnn_model.fit(validation_X, to_categorical(validation_Y), epochs=1, batch_size=256,)
performance = cnn_model.evaluate(validation_X, to_categorical(validation_Y))
# print("Accuracy on Validation samples: {0}".format(performance[1]))
performance = cnn_model.evaluate(test_X, to_categorical(test_Y))
print("Accuracy on Test samples: {0}".format(performance[1]))



# Feature design

In [ ]:
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, exposure

train_X,train_Y = preprocess_data(df_train)




In [ ]:
train_X1 = []
for image in train_X:
  image = image.reshape(48,48)
  fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualize=True)
  
  train_X1.append(hog_image.reshape(2304))
train_X1 = np.asarray(train_X1)
print(train_X1.shape)



In [ ]:
validation_X1 = []
for image in validation_X:
  image = image.reshape(48,48)
  fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualize=True)
  
  validation_X1.append(hog_image.reshape(2304))
validation_X1 = np.asarray(validation_X1)
print(validation_X1.shape)


In [ ]:
validation_X, validation_Y = preprocess_data(df_validation)

In [ ]:
model = Sequential([
  Dense(2304, input_shape=(48*48,), name="Input_layer"),
  # Dense(2304//2, activation='relu', name="second_hidden_layer"),Dropout(0.25),
  Dense(7, activation='softmax'),
])
print(model.summary())


# compile model
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)
start = time()
performance = model.fit(train_X1, to_categorical(train_Y), epochs=10, batch_size=256,)
print('Training time for this model is = ', time()-start)



In [ ]:
model.evaluate(validation_X1,to_categorical(validation_Y))